### Challenge
Monitorear constantemente el tipo de cambio entre el dolar americano (USD) y el peso Mexicano (MXN) para llenar una tabla de tipo de cambio y tener una estadística de como se comporta a lo largo del tiempo.

#### Importar librerias

In [4]:
import requests # Hace solicitudes HTTP y obtiene contenido de páginas web.
from bs4 import BeautifulSoup # Analiza y extrae datos de documentos HTML/XML que fueron obtenidos por medio de requests
import psycopg2 # Para conectarnos a Postgre y ejecutar consultas SQL.
from configparser import ConfigParser # Para leer y manejar los archivos config.ini
import logging #Para revisar el progreso y gestionar los errores que van apareciendo durante la ejecución del flujo
from datetime import datetime
import pandas as pd

#### Configurar logs

In [5]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

#### Creación de clases y métodos

Clase 1: Realiza la conexión a la página web y obtiene el tipo de cambio utilizando las librerias 'requests' y 'BeautifulSoup'

In [6]:
class ConnectionToWeb:
    def __init__(self,url):
        self.url = 'https://www.exchange-rates.org/es/conversor/usd-mxn'

    def get_exchange_rate (self):
        try:
            response = requests.get(self.url) # Realiza la solicitud (GET) a la URL
            response.raise_for_status() # Verifica el estado de la solicitud 
            
            soup = BeautifulSoup(response.content, 'html.parser') # Analiza el contenido HTML que devolvió la solicitud HTTP
            #Las siguientes 4 lineas buscan los atributos 'class' dentro de las etiquetas HTML
            from_amount = soup.find('span', class_='from-amount').text.strip() 
            from_currency = soup.find('span', class_='from-cn').text.strip()
            to_rate = soup.find('span', class_='to-rate').text.strip().replace(",", "")
            to_currency = soup.find('span', class_='to-cn').text.strip()

            update_date_str = soup.find('span', class_='converter-last-updated-value').text.strip()
            update_date = datetime.strptime(update_date_str, '%d/%m/%Y %H:%M %Z')

            logger.info(f'Tipo de cambio obtenido: {from_amount} {from_currency}= {to_rate} {to_currency}, última actualización: {update_date}')
            return from_amount, from_currency, to_rate, to_currency, update_date
        
        except Exception as e:
            logger.error (f'Error al obtener el tipo de cambio: {e}')
            raise

url = 'https://www.exchange-rates.org/es/conversor/usd-mxn'

web_scraping = ConnectionToWeb(url)
from_amount, from_currency, to_rate, to_currency, update_date = web_scraping.get_exchange_rate()

print(f'Tipo de cambio obtenido: {from_amount} {from_currency} = {to_rate} {to_currency}, última actualización: {update_date}')


2024-06-30 09:11:00,782 - INFO - Tipo de cambio obtenido: 1 dólar estadounidense= 18339 pesos mexicanos, última actualización: 2024-06-30 12:00:00


Tipo de cambio obtenido: 1 dólar estadounidense = 18339 pesos mexicanos, última actualización: 2024-06-30 12:00:00


Clase 2: Realiza todo el flujo para conectarse a la base de datos en PostgreSQL y hacer la apertura y cierre de la conexión de manera segura

In [15]:
class DatabaseManager:
    def __init__(self, config_file='config.ini'):
        self.config = self.read_config(config_file)
        self.conn = self.connect_db()

    def read_config(self, filename):
        config = ConfigParser()
        config.read(filename)
        if 'database' not in config:
            raise KeyError('La sección [database] no se encuentra en el archivo de configuración.')
        return config['database']

    def connect_db(self):
        try:
            conn = psycopg2.connect(
                host=self.config['host'],
                database=self.config['database'],
                user=self.config['user'],
                password=self.config['password']
            )
            logger.info("Conexión a la base de datos exitosa")
            return conn
        except Exception as e:
            logger.error(f"No se pudo conectar a la base de datos: {e}")
            raise

    def close_db_connection(self):
        if self.conn:
            self.conn.close()
            logger.info("Conexión a la base de datos cerrada")


Clase 3: En esta clase se realiza la creacion de la tabla y la insercion de datos en la misma

In [16]:
class TableManager:
    def __init__(self, db_manager):
        self.db_manager = db_manager

    def create_table(self):
        create_table_query = """
        CREATE TABLE IF NOT EXISTS currency (
            fromCurrency VARCHAR(50),
            toCurrency VARCHAR(50),
            total_value FLOAT,
            storeday TIMESTAMP
        );
        """
        try:
            cursor = self.db_manager.conn.cursor()
            cursor.execute(create_table_query)
            self.db_manager.conn.commit()
            logger.info("Tabla 'currency' creada exitosamente")
        except Exception as e:
            self.db_manager.conn.rollback()
            logger.error(f"Error al crear la tabla 'currency': {e}")
        finally:
            cursor.close()

    def insert_data(self, from_currency, to_currency, total_value, store_day):
        try:
            cursor = self.db_manager.conn.cursor()
            insert_query = """
            INSERT INTO currency (fromCurrency, toCurrency, total_value, storeday)
            VALUES (%s, %s, %s, %s);
            """
            cursor.execute(insert_query, (from_currency, to_currency, total_value, store_day))
            self.db_manager.conn.commit()
            logger.info("Datos insertados correctamente en la base de datos")
        except Exception as e:
            self.db_manager.conn.rollback()
            logger.error(f"No se pudo insertar datos en la base de datos: {e}")
        finally:
            cursor.close()


Clase 4: Se encarga de coordinar el proceso de obtención y almacenamiento del tipo de cambio

In [17]:
class CurrencyMonitor:
    def __init__(self, url, config_file='config.ini'):
        self.web_scraper = ConnectionToWeb(url) # instancia de connectiontoweb
        self.db_manager = DatabaseManager(config_file) # instancia de databasemanager
        self.table_manager = TableManager (self.db_manager)

    def run(self):
        try:
            from_amount, from_currency, to_rate, to_currency, update_date = self.web_scraper.get_exchange_rate()
            self.table_manager.create_table()
            self.table_manager.insert_data(from_currency, to_currency, to_rate, update_date)
        except Exception as e:
            logger.error(f'Error en el proceso: {e}')
        finally:
            self.db_manager.close_db_connection()

Script principal para ejecutar todo el proceso

In [29]:
if __name__ == "__main__":
    url = 'https://www.exchange-rates.org/es/conversor/usd-mxn'
    monitor = CurrencyMonitor(url)
    monitor.run()

2024-06-30 11:28:58,266 - INFO - Conexión a la base de datos exitosa
2024-06-30 11:28:58,718 - INFO - Tipo de cambio obtenido: 1 dólar estadounidense= 18339 pesos mexicanos, última actualización: 2024-06-30 13:55:00
2024-06-30 11:28:58,761 - INFO - Tabla 'currency' creada exitosamente
2024-06-30 11:28:58,795 - INFO - Datos insertados correctamente en la base de datos
2024-06-30 11:28:58,797 - INFO - Conexión a la base de datos cerrada


### Consultas SQL y visualización de la tabla

In [30]:
config = ConfigParser()
config.read('config.ini')

conn_params = {
    'host': config['database']['host'],
    'database': config['database']['database'],
    'user': config['database']['user'],
    'password': config['database']['password']
}

conn = psycopg2.connect(**conn_params)
print("Conexión exitosa a la base de datos")

select_query = 'SELECT * FROM currency;'

try:
    cursor = conn.cursor()
    cursor.execute(select_query)

    col_names = [desc[0] for desc in cursor.description]
    rows = cursor.fetchall()

    col_widths = [len(col) for col in col_names]
    for row in rows:
        for i, val in enumerate(row):
            col_widths[i] = max(col_widths[i], len(str(val)))

    header_row = " | ".join(col.ljust(col_widths[i]) for i, col in enumerate(col_names))
    print(header_row)
    print("-" * len(header_row))

    for row in rows:
        from_currency = "USD" if row[0] == "dólar estadounidense" else row[0]
        to_currency = "MXN" if row[1] == "pesos mexicanos" else row[1]
        print(f"{from_currency.ljust(12)} | {to_currency.ljust(10)} | {row[2]} | {row[3]}")

except Exception as e:
    print(f"Error al ejecutar la consulta SQL: {e}")

finally:
    cursor.close()
    conn.close()


Conexión exitosa a la base de datos
fromcurrency         | tocurrency      | total_value  | storeday  
------------------------------------------------------------------
USD          | MXN        | 18339.000000 | 2024-06-29
USD          | MXN        | 18339.000000 | 2024-06-29
USD          | MXN        | 18339.000000 | 2024-06-30
USD          | MXN        | 18339.000000 | 2024-06-30
USD          | MXN        | 18339.000000 | 2024-06-30
